# Neural Networks Final Project
### Reimplementation of the study: <br> ***"DE-FAKE: Detection and Attribution of Fake Images Generated by Text-to-Image GenerationModels"* <br> from Zeyang Sha, Zheng Li, Ning Yu, Yang Zhang**

**Name**: *Laura Papi*

**Matricola**: *1760732*

# Project Description

The above cited study focuses on the growing concerns about the possible misuse of AI generated images, and assesses the necessity for a tool to detect, and attribute, these fake images.<br>
In particular, it points out the lack of research on the particular case of images generated by a text prompt.
<br>

<br>
Therefore, this research proposes methods to answer the following 3 research questions [RQ]:

- **RQ1**. Detection of images generated by text-to-image generation models

- **RQ2**. Attribution of the fake images to their source model

- **RQ3**. Analysis of the likelihood that different text prompts have to generate authentic images

<br><br>
The following sections contain examples for my implementation of the described methods.<br><br>
The complete implementation of the models can be found in the source directory of the GitHub repository __[Source Code](http://url)__


## RQ1. Detection of images generated by text-to-image generation models

The study proposes two detector models:

1. **Image-only detector**<br>binary classifier that decides whether an input image is fake or real.

2. **Hybrid detector**<br>binary classifier that is able to tell if an image is fake or real, based on the input image and its corresponding text prompt.


### 1. Image-only detector

#### 1.1 Dataset
All the datasets are constitueted by a set of N real images (labeled 1), and a set of N corresponding fake generated images (labeled 0).

##### 1.1.1 Data Collection
The data used for the training is collected and generated as described in the following steps **(i)** and **(ii)**

- **(i)** Real images are fetched from the MSCOCO dataset, together with their captions.

In [ ]:
import sys
import os
#import the path to the scripts needed for this section
sys.path.insert(10, '/home/parwal/Documents/GitHub/De-Fake_nn_final_project/src/imageonly_detector')
#TODO capire a chi serve questo import e metterlo nel posto giusto

from src.imageonly_detector.MSCOCO_data_collection import fetchImagesFromMSCOCO

#SD+MSCOCO
fetchImagesFromMSCOCO("data/MSCOCO_for_SD/images", "data/MSCOCO_for_SD", 100)

#LD+MSCOCO --------------------------------------------------------------------------
fetchImagesFromMSCOCO("data/MSCOCO_for_LD/images", "data/MSCOCO_for_LD", 50)

#GLIDE+MSCOCO -----------------------------------------------------------------------
fetchImagesFromMSCOCO("data/MSCOCO_for_GLIDE/images", "data/MSCOCO_for_GLIDE", 50)

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO

- **(ii)** The captions from the MSCOCO images are used as input to the Stable Diffusion (SD) text-to-image generator.

In [ ]:
#SD+MSCOCO --------------------------------------------------------------------------
#use stable-diffusion API to generate 100 fake images from the 100 captions collected before
#prima di eseguire il file ho cambiato le directory
%run src/imageonly_detector/SD_MSCOCO_data_generation.py

#LD+MSCOCO --------------------------------------------------------------------------
#resetto la directory corrente a quella del progetto de-fake, altrimenti il file da eseguire non viene trovato
#questo è necessario perché LD_MSCOCO_data_generation.py cambia la directory a quella di latent-diffusion
os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")
%run src/imageonly_detector/LD_MSCOCO_data_generation.py

#GLIDE+MSCOCO -----------------------------------------------------------------------
#NON HO MAI PROVATO A RUNNARLO, altrimenti rigenera il modello (3gb)
#provare a runnarlo proprio alla fine di tutto per sicurezza
%run src/imageonly_detector/GLIDE_MSCOCO_data_generation.ipynb

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO (è a pagamento soltanto con le API, valutare)

##### 1.1.2 Dataset Construction
The collected and generated data is then shaped in the following structure, in order to be used for the training and evaluation step:<br><br>
train/<br>
    ├── class_0/<br>
    │   ├── ...<br>
    │   └── all the fake images<br>
    ├── class_1/<br>
    │   ├── ...<br>
    │   └── all the real images<br>
val/<br>
    ├── class_0/<br>
    │   ├── ...<br>
    │   └── all the fake images<br>
    ├── class_1/<br>
    │   ├── ...<br>
    │   └── all the real images<br>

In [ ]:
#transform the collected data in the previously described structure

from src.imageonly_detector.format_dataset import formatIntoDataset, formatIntoTrainTest

#SD+MSCOCO
#this function generates a pair of datasets (train and val), starting from data from the Stable Diffusion generation
#the data generated from SD contains 100 images, this original dataset is split in half (50 for train, 50 for test)
formatIntoTrainTest("data/MSCOCO_for_SD/images", "data/SD+MSCOCO/images", "data/imageonly_detector_data")
print("ok SD")

#LD+MSCOCO --------------------------------------------------------------------------
formatIntoDataset("data/MSCOCO_for_LD/images", "../latent-diffusion/outputs/txt2img-samples", "data/imageonly_detector_data/val_LD")
print("ok LD")

#GLIDE+MSCOCO -----------------------------------------------------------------------
formatIntoDataset("data/MSCOCO_for_GLIDE/images", "data/GLIDE+MSCOCO/images", "data/imageonly_detector_data/val_GLIDE")
print("ok GLIDE")

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO (è a pagamento soltanto con le API, valutare)

#### 1.2 Detector

The model is defined and trained in the file executed in the followind code block.

In [ ]:
#this function trains the model and tests it at every epoch
#both the test and train datasets are generated using SD
%run src/imageonly_detector/train.py

### 2. Hybrid detector

#### 2.1 Dataset

The dataset is built in the exact same way as the dataset for the image-only detector.
The following are the instructions to run in order to build:
- one training dataset (using images generated from SD)
- three evaluation dataset (using images generated from SD, LD and GLIDE respectively)

In [ ]:
import sys
import os

# N.B.
# before running this block you need to erase all the content of the following directories:
# data/MSCOCO_for_SD
# data/MSCOCO_for_LD
# data/MSCOCO_for_GLIDE
# data/SD+MSCOCO
# data/GLIDE+MSCOCO
# latent-diffusion/outputs/txt2img-samples

# ------------------- COLLECT REAL IMAGES FROM MSCOCO -------------------- #
#import the path to the scripts needed for this section
sys.path.insert(10, '/home/parwal/Documents/GitHub/De-Fake_nn_final_project/src/imageonly_detector')
#TODO capire a chi serve questo import e metterlo nel posto giusto

from src.imageonly_detector.MSCOCO_data_collection import fetchImagesFromMSCOCO

#SD+MSCOCO
fetchImagesFromMSCOCO("data/MSCOCO_for_SD/images", "data/MSCOCO_for_SD", 100)

#LD+MSCOCO --------------------------------------------------------------------------
fetchImagesFromMSCOCO("data/MSCOCO_for_LD/images", "data/MSCOCO_for_LD", 50)

#GLIDE+MSCOCO -----------------------------------------------------------------------
fetchImagesFromMSCOCO("data/MSCOCO_for_GLIDE/images", "data/MSCOCO_for_GLIDE", 50)

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO

In [ ]:
import os
# ------------------- GENERATE FAKE IMAGES USING SD, LD, GLIDE -------------------- #
#SD+MSCOCO --------------------------------------------------------------------------
#use stable-diffusion API to generate 100 fake images from the 100 captions collected before
%run src/imageonly_detector/SD_MSCOCO_data_generation.py

#LD+MSCOCO --------------------------------------------------------------------------
# N.B.
# prima di lanciare questo comando, aggiungere il file src/imageonly_detector/txt2img_batch.py alla directory latent-diffusion/scripts/
#resetto la directory corrente a quella del progetto de-fake, altrimenti il file da eseguire non viene trovato
#questo è necessario perché LD_MSCOCO_data_generation.py cambia la directory a quella di latent-diffusion
os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")
%run src/imageonly_detector/LD_MSCOCO_data_generation_batch.py

#GLIDE+MSCOCO -----------------------------------------------------------------------
#NON HO MAI PROVATO A RUNNARLO, altrimenti rigenera il modello (3gb)
#provare a runnarlo proprio alla fine di tutto per sicurezza
%run src/imageonly_detector/GLIDE_MSCOCO_data_generation.ipynb #TODO

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO (è a pagamento soltanto con le API, valutare)

In [ ]:
# ------------------- FORMAT THE DATA INTO THE STRUCTURE NEEDED FOR TRAINING/TESTING -------------------- #
os.chdir("/home/parwal/Documents/GitHub/De-Fake_nn_final_project")

#transform the collected data in the previously described structure
from src.imageonly_detector.format_dataset import formatIntoDataset, formatIntoTrainTest


#SD+MSCOCO --------------------------------------------------------------------------
#this function generates a pair of datasets (train and val), starting from data from the Stable Diffusion generation
#the data generated from SD contains 100 images, this original dataset is split in half (50 for train, 50 for test)
formatIntoTrainTest("data/MSCOCO_for_SD/images", "data/SD+MSCOCO/images", "data/hybrid_detector_data")
print("ok SD")

#LD+MSCOCO --------------------------------------------------------------------------
formatIntoDataset("data/MSCOCO_for_LD/images", "../latent-diffusion/outputs/txt2img-samples", "data/hybrid_detector_data/val_LD")
print("ok LD")

#GLIDE+MSCOCO -----------------------------------------------------------------------
formatIntoDataset("data/MSCOCO_for_GLIDE/images", "data/GLIDE+MSCOCO/images", "data/hybrid_detector_data/val_GLIDE") #TODO
print("ok GLIDE")

#DALL-E2+MSCOCO ---------------------------------------------------------------------
#TODO (è a pagamento soltanto con le API, valutare)

#### 2.2 Detector

The model structure is defined in src/hybrid_detector/hybrid_detector.py<br><br>

The training dataset is generated using Stable Diffusion (SD), same as for the image-only detector.<br>
It can be trained running the following command:

In [ ]:
# this function first builds the dataloader for the training datasert
# then it trains the model and saves the trained weights in the directory trained_models/
%run src/hybrid_detector/train.py

Now that the model is trained, we can evaluate it on some test datasets.<br>
In particular we will evaluate it on:<br>
- Stable Diffusion (SD), dataset generated from the same image-to-text generator used for the train dataset.
- GLIDE
- Latent Diffusion

In [ ]:
# encode the data into dataloaders suitable for the model, and run the evaluation algorithm.
%run src/hybrid_detector/eval.py

## RQ2. Attribution of the fake images to their source model

The study proposes two attributor models:

1. **Image-only attributor**<br>multi-class classifier that assigns each input image to its source generation model.

2. **Hybrid attributor**<br>multi-class classifier that assigns each input image to its source generation model, based on the input image and its corresponding text prompt.


### 1. Image-only attributor

#### 1.1 Dataset

The dataset is built similarly as in the previous cases, using fake images generated by text-to-image generation models.
But in this case the goal is to train a model to classify fake images based on the model that generated them.
So we will need to build a multi-class classifier, with each class corresponding to one model.

This means that the dataset we are going to build is obtained by:
- retrieving N captions from MSCOCO
- using one third of the propmts per each model, to get N/3 fake images from each model

This will lead us to build a dataset with the following structure:

attributor_dataset/<br>
    ├── class_SD/<br>
    │   ├── ...<br>
    │   └── all the images generated by SD<br>
    ├── class_GLIDE/<br>
    │   ├── ...<br>
    │   └── all the images generated by GLIDE<br>
    ├── class_LD/<br>
    │   ├── ...<br>
    │   └── all the images generated by LD<br>

In [ ]:
from src.imageonly_attributor.format_dataset import format_dataset_multiclass
from src.imageonly_attributor.dataset_generator import SD_generation, LD_generation, GLIDE_generation
from src.imageonly_detector.MSCOCO_data_collection import fetchCaptionsFromMSCOCO
import os

proj_dir = "../De-Fake_nn_final_project"
os.chdir(proj_dir)

# fetch the captions (N=150)
fetchCaptionsFromMSCOCO("data/imageonly_attributor_data", 50) # TODO fix chiama la funzione giusta con i parametri giusti

# use the first 50 (N/3) captions to generate images with SD
SD_generation("data/imageonly_attributor_data/train/mscoco_captions.csv", "data/imageonly_attributor_data/generated/SD+MSCOCO")

# use other 50 (N/3) captions to generate images with GLIDE
GLIDE_generation("data/imageonly_attributor_data/train/mscoco_captions.csv", "data/imageonly_attributor_data/generated/GLIDE+MSCOCO")

# use the last 50 (N/3) captions to generate images with LD
LD_generation("data/imageonly_attributor_data/train/mscoco_captions.csv")

# move the generated images to the dataset dir
# TODO trasforma tutte le stringhe in variabili che puo settare l'utente, con istruzioni su come settarle
format_dataset_multiclass("data/imageonly_attributor_data/generated/SD+MSCOCO", "../latent-diffusion/outputs/txt2img-samples", "data/imageonly_attributor_data/generated/GLIDE+MSCOCO", "data/imageonly_attributor_data/train")

In [ ]:
# Repeat the same procedure for the test dataset

from src.imageonly_attributor.format_dataset import format_dataset_multiclass
from src.imageonly_attributor.dataset_generator import SD_generation, LD_generation
from src.imageonly_detector.MSCOCO_data_collection import fetchImagesFromMSCOCO

# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/imageonly_attributor_data/test/class_real", "data/imageonly_attributor_data/test", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/imageonly_attributor_data/test/mscoco_captions.csv", "data/imageonly_attributor_data/generated/SD+MSCOCO_test")

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/imageonly_attributor_data/test/mscoco_captions.csv", "data/imageonly_attributor_data/test/GLIDE+MSCOCO_test")

# use the same 50 captions to generate images with LD OK
LD_generation("data/imageonly_attributor_data/test/mscoco_captions.csv")

# move the generated images to the dataset dir
# TODO trasforma tutte le stringhe in variabili che puo settare l'utente, con istruzioni su come settarle
format_dataset_multiclass("data/imageonly_attributor_data/generated/SD+MSCOCO_test", "../latent-diffusion/outputs/txt2img-samples", "data/imageonly_attributor_data/generated/GLIDE+MSCOCO", "data/imageonly_attributor_data/test")

#### 1.2 Attributor

In [ ]:
#this function trains the model and tests it at every epoch
#both the test and train datasets are generated using MSCOCO, SD, LD, GLIDE
%run src/imageonly_detector/train.py

### 2. Hybrid attributor

#### 2.1 Dataset

In [ ]:
from src.imageonly_attributor.format_dataset import format_dataset_multiclass
from src.imageonly_attributor.dataset_generator import SD_generation, LD_generation, GLIDE_generation
from src.imageonly_detector.MSCOCO_data_collection import fetchImagesFromMSCOCO

# fetch the images with their captions from MSCOCO (N=50)
#fetchImagesFromMSCOCO("data/hybrid_attributor_data/train/class_real", "data/hybrid_attributor_data/train", 50)

# use the same 50 captions to generate images with SD
#SD_generation("data/hybrid_attributor_data/train/mscoco_captions.csv", "data/hybrid_attributor_data/generated/SD+MSCOCO_train")

# use the same 50 captions to generate images with GLIDE
#GLIDE_generation("data/hybrid_attributor_data/train/mscoco_captions.csv", "data/hybrid_attributor_data/generated/GLIDE+MSCOCO_train")

# use the same 50 captions to generate images with LD OK
LD_generation("data/hybrid_attributor_data/train/mscoco_captions.csv")

# move the generated images to the dataset dir
# TODO trasforma tutte le stringhe in variabili che puo settare l'utente, con istruzioni su come settarle
format_dataset_multiclass("data/hybrid_attributor_data/generated/SD+MSCOCO_train", "../latent-diffusion/outputs/txt2img-samples", "data/hybrid_attributor_data/generated/GLIDE+MSCOCO_train", "data/hybrid_attributor_data/train")

In [ ]:
from src.imageonly_attributor.format_dataset import format_dataset_multiclass
from src.imageonly_attributor.dataset_generator import SD_generation, LD_generation, GLIDE_generation
from src.imageonly_detector.MSCOCO_data_collection import fetchImagesFromMSCOCO

# fetch the images with their captions from MSCOCO (N=50)
fetchImagesFromMSCOCO("data/hybrid_attributor_data/test/class_real", "data/hybrid_attributor_data/test", 50)

# use the same 50 captions to generate images with SD
SD_generation("data/hybrid_attributor_data/test/mscoco_captions.csv", "data/hybrid_attributor_data/generated/SD+MSCOCO_test")

# use the same 50 captions to generate images with GLIDE
GLIDE_generation("data/hybrid_attributor_data/test/mscoco_captions.csv", "data/hybrid_attributor_data/generated/GLIDE+MSCOCO_test")

# use the same 50 captions to generate images with LD OK
LD_generation("data/hybrid_attributor_data/test/mscoco_captions.csv")

# move the generated images to the dataset dir
# TODO trasforma tutte le stringhe in variabili che puo settare l'utente, con istruzioni su come settarle
format_dataset_multiclass("data/hybrid_attributor_data/generated/SD+MSCOCO_test", "../latent-diffusion/outputs/txt2img-samples", "data/hybrid_attributor_data/generated/GLIDE+MSCOCO_test", "data/hybrid_attributor_data/test")

#### 2.2 Attributor

## RQ3. Analysis of the likelihood that different text prompts have to generate authentic images

### 1. Semantic Analysis

### 2. Structure Analysis

## Conclusions